## Imports

In [200]:
# Read stocks
import yfinance as yf

import pandas as pd

# To read external property file
from jproperties import Properties

## Load External Configurations

In [201]:
configs = Properties()

with open('config/yf_stock_picker_p2.properties', 'rb') as config_file:
     configs.load(config_file)

SYMBOLS = configs.get('SYMBOLS').data.split(',') 
METRICS = configs.get('METRICS').data.split(',')
WEIGHTS = configs.get('WEIGHTS').data.split(',')

# Convert weights to integers
WEIGHTS = [int(i) for i in WEIGHTS]

## Utility method to compute Opearting Margin

In [202]:
# Computes operating margin
# ticker - ticker symbol to calculate operating income
def compute_om(ticker):
    income_stmt_df = ticker.income_stmt
    # Check to see whether Operating Income exists in the income statement first
    if 'Operating Income' in income_stmt_df.index:
        op_income = income_stmt_df.loc['Operating Income'][0]
        tot_revenue = income_stmt_df.loc['Total Revenue'][0]
        op_margin = op_income/tot_revenue
    else:
        # No 'Operating Income' found, use the operating margin from the info section
        op_margin = ticker.info['operatingMargins']
    return op_margin

## Utility method to return the ranking position

In [203]:
# Returns the ranking; -1 if the given ticker is not found in the metric
# raw_dict - dictionary of values to rank; dictionary -> symbol, value
# order - True for descending order or False for ascending
def get_ranking(raw_dict, order, ticker):
    # Rank the dictionary values in ascending/descending order
    ranked_dict = dict(sorted(raw_dict.items(), key=lambda item: item[1], reverse=order))
    if ticker in ranked_dict:
        # Returns rank/out of format, e.g. 7/18
        return f'{(list(ranked_dict).index(ticker)) + 1}/{len(ranked_dict)}'
    # No ranking found, due to a metric is not found with the ticker
    return 'NA'

## Ranking List for given Metric

In [204]:
# Returns the ranking list for given metric
# metric_name - name of the metric
# order - order for ranking, default is True, i.e., descending
def get_ranking_list(metric_dict, order):
    ranking_list = []
    for symbol in SYMBOLS:
        ranking_list.append(get_ranking(metric_dict, order, symbol))
    return ranking_list

## Collect Key Metrics data

In [205]:
# DF to collect exclusions
key_metrics_ex_df = pd.DataFrame()
# Initialize list with empty dictionaries. This collects raw metrics data for each metric
raw_metrics = [{} for sub in range(len(METRICS))]

for symbol in SYMBOLS:
    ticker = yf.Ticker(symbol)
    # Calculate Operating Margin and convert it to percentage
    raw_metrics[0][symbol] = round((compute_om(ticker) * 100), 2)

    # Dividend Yield -> Check to see whether the dividend yield exists for the symbol
    if 'dividendYield' in ticker.info:
        raw_metrics[1][symbol] = round((ticker.info['dividendYield'] * 100), 2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[1], 'Reason':'Missing Dividend Yield'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    # Dividend Cover -> Check to see whether the dividend rate exists for the symbol
    if 'dividendRate' in ticker.info:
        raw_metrics[2][symbol] = round((ticker.info['trailingEps']/ticker.info['dividendRate']), 2)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[2], 'Reason':'Missing Dividend Rate'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    # Debt/EBITDA; check it whether total debt exists in income statement
    if 'Total Debt' in ticker.balance_sheet.index:
        debt_to_ebitda = round((ticker.balance_sheet.loc['Total Debt'][0]/ticker.income_stmt.loc['EBITDA'][0]), 2)
        # We only take into ccount with positive ratios or else they will impact rankings
        if debt_to_ebitda >= 0:
            raw_metrics[3][symbol] = debt_to_ebitda
        else:
            new_row = {'Symbol':symbol, 'Metric':METRICS[3], 'Reason':'Negative Debt/EBITDA ratio'}
            key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[3], 'Reason':'Missing Total Debt'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)

    # Fwd P/E -> Check to see whether the Fwd P/E exists for the symbol
    if 'forwardPE' in ticker.info:
        fwd_pe = ticker.info['forwardPE']
        if  fwd_pe >= 0:
            raw_metrics[4][symbol] = round(fwd_pe, 2)
        else:
            new_row = {'Symbol':symbol, 'Metric':METRICS[4], 'Reason':'Negaive Forward P/E ratio'}
            key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)        
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[4], 'Reason':'Missing Forward P/E ratio'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)
    
    # PEG Ration -> Check to see whether the PEG ration exists for the symbol
    if 'pegRatio' in ticker.info:
        peg_ratio = ticker.info['pegRatio']
        if peg_ratio >= 0:
            raw_metrics[5][symbol] = peg_ratio
        else:
            new_row = {'Symbol':symbol, 'Metric':METRICS[5], 'Reason':'Negaive PEG ratio'}
            key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)                    
    else:
        new_row = {'Symbol':symbol, 'Metric':METRICS[5], 'Reason':'Missing PEG Ratio'}
        key_metrics_ex_df = pd.concat([key_metrics_ex_df, pd.DataFrame([new_row])], ignore_index=True)
print('Done')

Done


## Populate Key Metrics

In [206]:
# DF to collect value key metrics for all the tickers
key_metrics_value_df = pd.DataFrame()

for idx in range(len(METRICS)):
    key_metrics_value_df[METRICS[idx]] = raw_metrics[idx]
key_metrics_value_df

,Operating Margin,Dividend Yield,Dividend Cover,Debt/EBITDA,P/E ratio,PEG ratio
INTU,21.86,0.70,2.34,1.63,27.43,2.13
CDNS,30.15,NaN,NaN,0.62,40.05,2.57
WDAY,-3.57,NaN,NaN,22.86,32.50,1.29
ROP,28.38,0.56,4.01,3.14,27.07,2.82
TEAM,-9.77,NaN,NaN,NaN,69.97,NaN
ADSK,19.76,NaN,NaN,2.34,24.40,1.87
DDOG,-3.50,NaN,NaN,86.12,55.54,2.09
ANSS,28.69,NaN,NaN,1.23,30.74,3.11
ZM,5.59,NaN,NaN,0.22,15.30,73.15
PTC,25.01,NaN,NaN,2.54,28.06,3.28


## Display Key Metrics Exceptions

In [207]:
# key_metrics_ex_df.head()

## Calculate Scores for each Metric

In [208]:
# DF to hold key metrics scores
key_metrics_score_df = pd.DataFrame()

# Set the index to SYMBOLS
key_metrics_score_df.index = SYMBOLS

# Loop through raw data we collected under Collect Key Metrics section
for idx in range(len(raw_metrics)):
    if (idx >= 3):
        # Ranking is on ascending order (lower the better)
        ranking_list = get_ranking_list(raw_metrics[idx], False)
    else:
        # Ranking is on descending order (higher the better)
        ranking_list = get_ranking_list(raw_metrics[idx], True)

    # List of scores for a metric
    score = []
    # Loop through the ranking list to calculate the score
    for ranking in ranking_list:
        if ranking == 'NA':
            score.append(0.0)
            continue
        rank, total = tuple(map(int, ranking.split('/')))
        score.append(round(((total - rank) + 1)/total,2))

    # Append the Score to the metric name
    col_name = f'{METRICS[idx]} Score'
    key_metrics_score_df[col_name] = score
key_metrics_score_df

,Operating Margin Score,Dividend Yield Score,Dividend Cover Score,Debt/EBITDA Score,P/E ratio Score,PEG ratio Score
INTU,0.78,0.75,0.25,0.62,0.65,0.44
CDNS,1.00,0.00,0.00,0.81,0.29,0.38
WDAY,0.22,0.00,0.00,0.12,0.35,0.81
ROP,0.89,0.50,1.00,0.44,0.71,0.31
TEAM,0.17,0.00,0.00,0.00,0.06,0.00
ADSK,0.61,0.00,0.00,0.56,0.76,0.56
DDOG,0.28,0.00,0.00,0.06,0.18,0.50
ANSS,0.94,0.00,0.00,0.75,0.41,0.25
ZM,0.39,0.00,0.00,0.94,0.94,0.06
PTC,0.83,0.00,0.00,0.50,0.53,0.19


## Calculate Total Score

In [209]:
# List of total scores for each stock
total_score = []
for symbol in SYMBOLS:
    # Sum all scores across key metrics * weights and normalize the score out of 10
    total_score.append(round(((((key_metrics_score_df.loc[symbol] * WEIGHTS).sum())/sum(WEIGHTS)) * 10), 2))
key_metrics_score_df['Total Score'] = total_score
key_metrics_score_df

,Operating Margin Score,Dividend Yield Score,Dividend Cover Score,Debt/EBITDA Score,P/E ratio Score,PEG ratio Score,Total Score
INTU,0.78,0.75,0.25,0.62,0.65,0.44,5.82
CDNS,1.00,0.00,0.00,0.81,0.29,0.38,4.13
WDAY,0.22,0.00,0.00,0.12,0.35,0.81,2.50
ROP,0.89,0.50,1.00,0.44,0.71,0.31,6.42
TEAM,0.17,0.00,0.00,0.00,0.06,0.00,0.38
ADSK,0.61,0.00,0.00,0.56,0.76,0.56,4.15
DDOG,0.28,0.00,0.00,0.06,0.18,0.50,1.70
ANSS,0.94,0.00,0.00,0.75,0.41,0.25,3.92
ZM,0.39,0.00,0.00,0.94,0.94,0.06,3.88
PTC,0.83,0.00,0.00,0.50,0.53,0.19,3.42


## Ranking using Total Score

In [210]:
# Each score in this list has the format: score/total, e.g. 4/17
tot_score_ranking = get_ranking_list(key_metrics_score_df['Total Score'], True)
# Only interested in the score part
ranking = [x.split('/')[0] for x in tot_score_ranking]
key_metrics_score_df['Ranking'] = ranking
key_metrics_score_df

,Operating Margin Score,Dividend Yield Score,Dividend Cover Score,Debt/EBITDA Score,P/E ratio Score,PEG ratio Score,Total Score,Ranking
INTU,0.78,0.75,0.25,0.62,0.65,0.44,5.82,3
CDNS,1.00,0.00,0.00,0.81,0.29,0.38,4.13,7
WDAY,0.22,0.00,0.00,0.12,0.35,0.81,2.50,15
ROP,0.89,0.50,1.00,0.44,0.71,0.31,6.42,2
TEAM,0.17,0.00,0.00,0.00,0.06,0.00,0.38,18
ADSK,0.61,0.00,0.00,0.56,0.76,0.56,4.15,6
DDOG,0.28,0.00,0.00,0.06,0.18,0.50,1.70,17
ANSS,0.94,0.00,0.00,0.75,0.41,0.25,3.92,8
ZM,0.39,0.00,0.00,0.94,0.94,0.06,3.88,9
PTC,0.83,0.00,0.00,0.50,0.53,0.19,3.42,11


In [211]:
## Debugging - to drop a column

In [212]:
# key_metrics_score_df.drop(columns=['Total Score'], inplace=True)